In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import DBSCAN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import IsolationForest 
from sklearn.feature_selection import mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

In [30]:
# Bootstrap pipeline with bagging classifier

kf = KFold(n_splits=5, shuffle = True)

c_values = np.array([0.8])
bandwidth = np.array([0.001])
BMAC_means_test = np.array([])
BMAC_stds_test = np.array([])
BMAC_means_train = np.array([])
BMAC_stds_train = np.array([])

XTRAIN = pd.read_csv("X_train.csv")
XTRAIN = XTRAIN.iloc[:, 1:]
XTEST = pd.read_csv("X_test.csv")
XTEST = XTEST.iloc[:, 1:]
YTRAIN = pd.read_csv("Y_train.csv")

# Remove id from ytrain
YTRAIN = YTRAIN.drop('id', axis = 1)

for g in bandwidth:
 for c in c_values:
        BMAC_scores_test = np.array([])
        BMAC_scores_train = np.array([])
        
        for train_index, test_index in kf.split(XTRAIN.values):
        
            x_train = XTRAIN.loc[train_index, :]
            x_test = XTRAIN.loc[test_index, :]

            y_train = YTRAIN.loc[train_index, :]
            y_test = YTRAIN.loc[test_index, :]

            #low variance 
            var = VarianceThreshold(threshold=0.3)
            var.fit(x_train)
            x_train = var.transform(x_train)
            x_test = var.transform(x_test)

            #standardise
            scaler = preprocessing.StandardScaler()
            x_train = scaler.fit_transform(x_train)
            x_test = scaler.fit_transform(x_test)

            #mutual information
            sel_mutual = SelectKBest(mutual_info_classif, k = 500)
            sel_mutual.fit(x_train, y_train)
            x_train = sel_mutual.transform(x_train)
            x_test = sel_mutual.transform(x_test)

            #outlier detection 
            clf = IsolationForest(n_estimators=200)
            clf.fit(x_train)
            clf.decision_function(x_train)
            inlier_indices = np.where(clf.predict(x_train) == 1)[0]
            print("Size of inliers:", inlier_indices.shape)
            x_train = pd.DataFrame(x_train).iloc[inlier_indices]
            y_train = pd.DataFrame(y_train).iloc[inlier_indices]
            print("Inliers shapes:", x_train.shape, x_train.shape)

            # Handle class imbalance with smote function
            sm = SMOTE(random_state=42)
            x_train, y_train = sm.fit_resample(x_train.values, y_train.values) #x_train and y_train are now arrays

            estimator = SVC(C=c, gamma = g)       
            estimator.fit(x_train, y_train.ravel())
            pred = estimator.predict(x_test)
            BMAC_test = balanced_accuracy_score(y_test, pred)
            BMAC_scores_test = np.append(BMAC_scores_test, BMAC_test)

            pred = estimator.predict(x_train)
            BMAC_train = balanced_accuracy_score(y_train, pred)
            BMAC_scores_train = np.append(BMAC_scores_train, BMAC_train)

            print('mean for current c=', c,'and g=', g, 'is, test:', np.mean(BMAC_scores_test), 'train:',  np.mean(BMAC_scores_train))


            BMAC_means_test = np.append(BMAC_means_test, np.mean(BMAC_scores_test))
            BMAC_stds_test = np.append(BMAC_stds_test, np.std(BMAC_scores_test))
            BMAC_means_train = np.append(BMAC_means_train, np.mean(BMAC_scores_train))
            BMAC_stds_train = np.append(BMAC_stds_train, np.std(BMAC_scores_train))
    

print(BMAC_means_test)
print(BMAC_stds_test)
print(BMAC_means_train)
print(BMAC_stds_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


Size of inliers: (3456,)
Inliers shapes: (3456, 500) (3456, 500)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mean for current c= 0.8 and g= 0.001 is, test: 0.6860912160404152 train: 0.8560424966799468


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


...

mean for current c= 0.8 and g= 0.001 is, test: 0.666395779501072 train: 0.8536520584329349


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


Size of inliers: (3456,)
Inliers shapes: (3456, 500) (3456, 500)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [19]:
      
x_train = pd.read_csv("X_train.csv")
x_train = x_train.iloc[:, 1:]
x_test = pd.read_csv("X_test.csv")
x_test = x_test.iloc[:, 1:]
y_train = pd.read_csv("Y_train.csv")

# Remove id from ytrain
y_train = y_train.drop('id', axis = 1)

sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x_train.values, y_train.values) #x_train and y_train are now arrays

# Scale the data (should this be done for each bootstrap sample? in that case how)
scaler = preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)
#x_train = pd.DataFrame(xtrain_scaled, columns = x_train.columns)
x_test = scaler.fit_transform(x_test)
#x_test = pd.DataFrame(xtest_scaled, columns = x_test.columns)

    
    
model = SVC(C=0.8, gamma = 0.001)
model.fit(x_train, y_train)
pred = model.predict(x_test)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: name 'csv' is not defined

In [20]:
index = pd.read_csv("sample.csv")
index['y'] = pred

index.to_csv("svm_tuned_for_loops.csv")